In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data

In [3]:
df = get_energy_data()
df = prepare_data(df)

  0%|          | 0/255 [00:00<?, ?it/s]/Users/elias/Desktop/PTSFC/Energy/HelpFunctions/get_energy_data.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 255/255 [00:29<00:00,  8.74it/s]


In [27]:
def remove_time(df, num_years = 0, num_months = 0, num_weeks = 0, num_days = 0, num_hours = 0):
    # date_to_keep = df.index.max() - num_years - num_months - num_days
    date_to_keep = df.index.max() - pd.DateOffset(years = num_years, months=num_months, weeks = num_weeks, days=num_days, hours = num_hours)
    df = df.loc[df.index <= date_to_keep]
    return df

In [17]:
df

,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0
...,...,...
2023-11-12 02:00:00,38.89100,6
2023-11-12 03:00:00,38.68725,6
2023-11-12 04:00:00,39.28950,6


,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0
...,...,...
2022-11-09 02:00:00,44.18275,2
2022-11-09 03:00:00,44.86825,2
2022-11-09 04:00:00,46.26975,2
